# **MACHINE LEARNING (AMOSTA 3 MILHÕES CORRIDAS):**

--------------------------------------------

- # **OBJETIVO 1**: Ensinar um modelo a prever a coluna trip_duration_minutes com base em outras características da corrida com PySpark MLlib

- Preparando Ambiente e carregando dados:

In [5]:
print("Passo 1: Instalando bibliotecas...")
!pip install pyspark google-cloud-bigquery pandas-gbq db-dtypes -q
print("--> Bibliotecas instaladas com sucesso.\n")
from pyspark.sql import SparkSession
from google.colab import auth

print("Passo 2: Autenticação de usuário...")
auth.authenticate_user()
print("--> Usuário autenticado com sucesso!\n")


print("Passo 3: Iniciando e configurando a sessão Spark...")
print("(Esta parte pode demorar um pouco enquanto baixa o conector)")
spark = SparkSession.builder \
    .appName("PySpark-BigQuery-Conexao-Nova") \
    .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.34.0") \
    .getOrCreate()
print("--> Sessão Spark pronta para uso!\n")

# Definição das variáveis e leitura dos dados do BigQuery
project_id = 'leanttro-projeto-taxi'
table_id = 'dados_analise.dados_limpos'

print(f"Passo 4: Carregando dados da tabela '{project_id}.{table_id}'...")
try:
    # Este é o código de leitura que você enviou, que está correto.
    # Ele vai usar a sessão 'spark' que acabamos de criar e configurar.
    df_silver_completo = spark.read.format('bigquery') \
      .option('table', f'{project_id}.{table_id}') \
      .option('parentProject', project_id) \
      .load() \
      .limit(3000000)

    # Coloca os dados em memória para acesso rápido
    df_silver_completo.cache()

    print("\n========================================")
    print(" ✅ SUCESSO! DADOS CARREGADOS!")
    print("========================================")
    print(f"Total de registros carregados: {df_silver_completo.count()}")
    print("\nSchema do DataFrame:")
    df_silver_completo.printSchema()

except Exception as e:
    print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(" ❌ FALHA AO CARREGAR OS DADOS.")
    print(" O erro foi:")
    print(e)
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

Passo 1: Instalando bibliotecas...
--> Bibliotecas instaladas com sucesso.

Passo 2: Autenticação de usuário...
--> Usuário autenticado com sucesso!

Passo 3: Iniciando e configurando a sessão Spark...
(Esta parte pode demorar um pouco enquanto baixa o conector)
--> Sessão Spark pronta para uso!

Passo 4: Carregando dados da tabela 'leanttro-projeto-taxi.dados_analise.dados_limpos'...

 ✅ SUCESSO! DADOS CARREGADOS!
Total de registros carregados: 3000000

Schema do DataFrame:
root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- pickup_day_of_week: long (nullable = true)
 |-- pickup_hour: long (nullable = true)
 |-- trip_duration_minutes: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- total_amount: double (nullable = true)



- Separando Colunas e Garantindo Limpeza:

In [6]:
df_modelo = df_silver_completo.select(
    "pickup_hour",
    "pickup_day_of_week",
    "trip_distance",
    "passenger_count",
    "trip_duration_minutes",
    "total_amount"
).na.drop()

- Dividindo os dados em treino (80%) e teste (20%):

In [7]:
(df_treino, df_teste) = df_modelo.randomSplit([0.8, 0.2], seed=42)

-  Colocando em cache novamente:

In [8]:
df_treino.cache()
df_teste.cache()
print(f"Dados prontos para o modelo. Treino: {df_treino.count()}, Teste: {df_teste.count()}")

Dados prontos para o modelo. Treino: 2399681, Teste: 600319


.cache(): É uma otimização do Spark para guardar esses dois conjuntos de dados na memória, tornando o acesso a eles mais rápido nas etapas seguintes de treinamento.

## Preparando as Features:

In [9]:
from pyspark.ml.feature import VectorAssembler
features_cols = ["pickup_hour", "pickup_day_of_week", "trip_distance", "passenger_count"]
assembler = VectorAssembler(inputCols=features_cols, outputCol="features")

# Transformando os dataframes de treino e teste:
df_treino_ml = assembler.transform(df_treino)
df_teste_ml = assembler.transform(df_teste)

# **MODELO DE ML:** Regressão Linear

- Carregar dados:


In [10]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
lr = LinearRegression(featuresCol="features", labelCol="trip_duration_minutes")
lr_modelo = lr.fit(df_treino_ml)


- # FAZENDO PREVISÕES:

In [11]:
previsoes_lr = lr_modelo.transform(df_teste_ml)

- Avaliando modelo:

In [12]:
evaluator = RegressionEvaluator(labelCol="trip_duration_minutes", predictionCol="prediction")
rmse_lr = evaluator.evaluate(previsoes_lr, {evaluator.metricName: "rmse"})
r2_lr = evaluator.evaluate(previsoes_lr, {evaluator.metricName: "r2"})

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- # Resultado do **Modelo LINEAR**:

In [13]:
print("\n--- Resultado da Regressão Linear ---")
print(f"RMSE: {rmse_lr:.2f} minutos")
print(f"R²: {r2_lr:.2%}")


--- Resultado da Regressão Linear ---
RMSE: 14.98 minutos
R²: -13.57%


RMSE: Ele pode errar até 14.98 min pra mais ou pra menos do que a duração da viagem!

Um R² negativo, como na amostra inferior, é considerado muito baixo. Ele está ativamente fazendo previsões ruins. POR ISSO, vamos treinar OUTRO modelo e avaliar novamente:

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# **MODELO DE ML:** Random Forest

n_estimators=50 significa que ele usará 50 "árvores de decisão" para votar na melhor previsão.
n_jobs=-1 usa todos os processadores disponíveis para acelerar o treinamento.

- Carregando dados:

In [14]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="trip_duration_minutes",
    seed=42,
    numTrees=50  # Equivalente ao n_estimators
)
rf_modelo = rf.fit(df_treino_ml)

- Fazendo previsões:

In [15]:
previsoes_rf = rf_modelo.transform(df_teste_ml)

- Avaliando o modelo (usando o mesmo 'evaluator' de antes):


In [16]:
rmse_rf = evaluator.evaluate(previsoes_rf, {evaluator.metricName: "rmse"})
r2_rf = evaluator.evaluate(previsoes_rf, {evaluator.metricName: "r2"})

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- # Resultado do **Modelo RANDOM Forest**:

In [17]:
print("\n--- Resultado do Random Forest ---")
print(f"RMSE: {rmse_rf:.2f} minutos")
print(f"R²: {r2_rf:.2%}")



--- Resultado do Random Forest ---
RMSE: 7.60 minutos
R²: 70.77%


%md
RMSE: Ele pode errar até 7.60 min pra mais ou pra menos do que a duração da viagem!

Um R² de 70.77% significa que ele pode acertar mais que 2/3 das previsões! Bem melhor do que o modelo de regressão linear para esse caso!

**OU SEJA, OBTEVE UM RESULTADO MELHOR QUE O PRIMEIRO TESTE!**

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- # **OBJETIVO 2**: Ensinar um modelo a prever a coluna total_amount com base nas características da corrida, incluindo a duração que nosso primeiro modelo previa usando PySpark MLlib

# **MODELO DE ML:** Random Forest Regressor


- Criando coluna "Duração Prevista":

In [18]:
df_treino_com_previsao_tempo = rf_modelo.transform(df_treino_ml) \
                                        .withColumnRenamed("prediction", "duracao_prevista")
df_teste_com_previsao_tempo = rf_modelo.transform(df_teste_ml) \
                                       .withColumnRenamed("prediction", "duracao_prevista")

print("Coluna 'duracao_prevista' criada com sucesso.")
df_treino_com_previsao_tempo.select("trip_duration_minutes", "duracao_prevista").show(5)

Coluna 'duracao_prevista' criada com sucesso.
+---------------------+-----------------+
|trip_duration_minutes| duracao_prevista|
+---------------------+-----------------+
|                    6|7.412289584410088|
|                    6|7.440934449947852|
|                    2|7.412289584410088|
|                    2|7.440934449947852|
|                    2|7.412289584410088|
+---------------------+-----------------+
only showing top 5 rows



- Criando Features:

In [19]:
from pyspark.ml.feature import VectorAssembler
features_valor_cols = [
    "pickup_hour",
    "pickup_day_of_week",
    "trip_distance",
    "passenger_count",
    "duracao_prevista"  # <<< A previsão do primeiro modelo entra aqui!
]
assembler_valor = VectorAssembler(inputCols=features_valor_cols, outputCol="features_valor")
df_treino_valor = assembler_valor.transform(df_treino_com_previsao_tempo)
df_teste_valor = assembler_valor.transform(df_teste_com_previsao_tempo)
print("Features para o modelo de valor preparadas.")

Features para o modelo de valor preparadas.


- Carrendo dados focado no valor (total_amount):

In [24]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
rf_valor = RandomForestRegressor(
    featuresCol="features_valor",
    labelCol="total_amount", # <-- Added the labelCol here
    seed=42,
    numTrees=50
)

- Treinando modelo:

In [25]:
# Treinar o modelo
modelo_valor = rf_valor.fit(df_treino_valor)
print("Treinamento concluído!")

# Fazer previsões no conjunto de teste
previsoes_valor = modelo_valor.transform(df_teste_valor)


Treinamento concluído!


- Avaliando o modelo:

In [26]:
evaluator_valor = RegressionEvaluator(labelCol="total_amount", predictionCol="prediction")
rmse_valor = evaluator_valor.evaluate(previsoes_valor, {evaluator_valor.metricName: "rmse"})
r2_valor = evaluator_valor.evaluate(previsoes_valor, {evaluator_valor.metricName: "r2"})

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

- # Resultado do **Modelo RANDOM Forest Regressor**:

In [27]:
print("\n--- Resultado do Modelo de Previsão de Valor ---")
print(f"RMSE: ${rmse_valor:.2f} DÓLARES")
print(f"R²: {r2_valor:.2%}")


--- Resultado do Modelo de Previsão de Valor ---
RMSE: $9.24 DÓLARES
R²: 83.69%


RMSE: O valor está $9,24 DÓLARES de diferença (pra mais ou pra menos) do valor real, considerando todas as variações que podem alterar no valor final

R²: 83.69% significa que o modelo  consegue captar bem a relação entre as variáveis (tempo, distância, hora, etc.) e o preço.

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# **------------ SIMULADOR PARA CORRIDA: VALORES E TEMPO ------------**
- # (AMOSTRA 3 MILHÕES DE CORRIDAS):

In [39]:
from pyspark.ml.feature import VectorAssembler

# Selecionar colunas e remover nulos
df_modelo =  df_silver_completo.select(
    "pickup_hour",
    "pickup_day_of_week",
    "trip_distance",
    "passenger_count",
    "trip_duration_minutes",
    "total_amount"
).na.drop()

# Preparar o assembler de features para o primeiro modelo
features_cols_tempo = ["pickup_hour", "pickup_day_of_week", "trip_distance", "passenger_count"]
assembler_tempo = VectorAssembler(inputCols=features_cols_tempo, outputCol="features")

# Transformar os dados e dividir em treino e teste
df_transformado_tempo = assembler_tempo.transform(df_modelo)
(df_treino_tempo, df_teste_tempo) = df_transformado_tempo.randomSplit([0.8, 0.2], seed=42)

# Colocar em cache para performance
df_treino_tempo.cache()
df_teste_tempo.cache()

print("Dados preparados e divididos para o modelo de DURAÇÃO.")

Dados preparados e divididos para o modelo de DURAÇÃO.


- Treinando Modelo TEMPO (Modelo Random Forest Regressor):

In [40]:
from pyspark.ml.regression import RandomForestRegressor

# Configurar o modelo
rf_tempo = RandomForestRegressor(
    featuresCol="features",
    labelCol="trip_duration_minutes",
    seed=42,
    numTrees=50
)

# Treinar o modelo
print("Treinando o modelo de previsão de tempo... (Isso pode levar alguns minutos)")
rf_modelo_tempo = rf_tempo.fit(df_treino_tempo)

# SALVAR O MODELO TREINADO (PASSO CRUCIAL)
rf_modelo_tempo.write().overwrite().save("./modelo_tempo_spark")

print("Modelo de DURAÇÃO treinado e salvo com sucesso na pasta './modelo_tempo_spark'!")

Treinando o modelo de previsão de tempo... (Isso pode levar alguns minutos)
Modelo de DURAÇÃO treinado e salvo com sucesso na pasta './modelo_tempo_spark'!


- Treinando Modelo VALOR (Modelo Randon Forest Regressor) :

In [41]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor

df_treino_com_previsao_tempo = rf_modelo_tempo.transform(df_treino_tempo) \
                                              .withColumnRenamed("prediction", "duracao_prevista")

features_cols_valor = [
    "pickup_hour",
    "pickup_day_of_week",
    "trip_distance",
    "passenger_count",
    "duracao_prevista" # Usando a previsão do primeiro modelo como feature
]
assembler_valor = VectorAssembler(inputCols=features_cols_valor, outputCol="features_valor")
df_treino_valor = assembler_valor.transform(df_treino_com_previsao_tempo)

rf_valor = RandomForestRegressor(
    featuresCol="features_valor",
    labelCol="total_amount", # Agora o alvo é o valor total
    seed=42,
    numTrees=50
)

print("Treinando o modelo de previsão de valor... (Isso também pode demorar)")
modelo_valor = rf_valor.fit(df_treino_valor)

#  SALVANDO O SEGUNDO MODELO TREINADO
modelo_valor.write().overwrite().save("./modelo_valor_spark")

print("Modelo de VALOR treinado e salvo com sucesso na pasta './modelo_valor_spark'!")

Treinando o modelo de previsão de valor... (Isso também pode demorar)
Modelo de VALOR treinado e salvo com sucesso na pasta './modelo_valor_spark'!


- Executando Simulador:

In [43]:
from pyspark.ml.regression import RandomForestRegressionModel
def simular_corrida_spark_completo(km, hora, dia_semana, passageiros=1):
    try:
        modelo_tempo = RandomForestRegressionModel.load("./modelo_tempo_spark")
        modelo_valor = RandomForestRegressionModel.load("./modelo_valor_spark")
    except Exception as e:
        print(f"Erro ao carregar os modelos. Certifique-se de que as pastas './modelo_tempo_spark' e './modelo_valor_spark' existem.")
        print(f"Detalhe do erro: {e}")
        return

    milhas = km * 0.621371
    schema_inicial = "pickup_hour INT, pickup_day_of_week INT, trip_distance DOUBLE, passenger_count INT"
    dados_nova_corrida = spark.createDataFrame(
        data=[(hora, dia_semana, milhas, passageiros)],
        schema=schema_inicial
    )

    # Transforma os dados de entrada usando o primeiro assembler (para o modelo de tempo)
    dados_para_prever_tempo = assembler.transform(dados_nova_corrida)
    # Faz a previsão do tempo e renomeia a coluna de previsão para 'duracao_prevista'
    df_com_duracao_prevista = modelo_tempo.transform(dados_para_prever_tempo).withColumnRenamed("prediction", "duracao_prevista")

    duracao_prevista = df_com_duracao_prevista.select("duracao_prevista").first()[0]

    # Transforma o DataFrame (que agora tem a 'duracao_prevista') usando o segundo assembler (para o modelo de valor)
    dados_para_prever_valor = assembler_valor.transform(df_com_duracao_prevista)

    # Faz a previsão do valor
    previsao_valor_df = modelo_valor.transform(dados_para_prever_valor)

    # Extrai o resultado da previsão de valor
    valor_previsto = previsao_valor_df.select("prediction").first()[0]


    print("\n--- Previsão da Corrida (PySpark) ---")
    print(f"Distância: {km:.2f} km")
    print(f"Hora: {hora}:00h")
    print(f"Dia da semana: {dia_semana}")
    print(f">> Tempo estimado: {duracao_prevista:.1f} min")
    print(f">> Valor estimado: ${valor_previsto:.2f}")


# --- Parte Interativa ---
# (Pede os dados para o usuário e chama a função completa)
try:
    print("\n--- Simulador de Corrida Interativo (PySpark) ---")
    km_usuario = float(input("Digite a distância da corrida em KM (ex: 12.5): "))
    hora_usuario = int(input("Digite a hora do dia (0 a 23): "))
    dia_semana_usuario = int(input("Digite o dia da semana (1=Dom, 2=Seg, ..., 7=Sáb): "))

    simular_corrida_spark_completo(km_usuario, hora_usuario, dia_semana_usuario)

except ValueError:
    print("\nErro: Entrada inválida.")
except NameError:
    print("\nErro: Os 'assemblers' não foram definidos. Execute o código de treinamento primeiro.")


--- Simulador de Corrida Interativo (PySpark) ---
Digite a distância da corrida em KM (ex: 12.5): 25
Digite a hora do dia (0 a 23): 23
Digite o dia da semana (1=Dom, 2=Seg, ..., 7=Sáb): 7

--- Previsão da Corrida (PySpark) ---
Distância: 25.00 km
Hora: 23:00h
Dia da semana: 7
>> Tempo estimado: 37.9 min
>> Valor estimado: $80.96
